<a href="https://colab.research.google.com/github/MHusni1604/Pengantar-Anlisis-Data-Panel/blob/main/Tugas_Minggu_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisis Dampak Trump Tariffs Terhadap Harga Saham LQ45
---
![Trump Tariffs](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a1/Trump_showing_a_chart_with_reciprocal_tariffs_%28cropped%29.jpg/500px-Trump_showing_a_chart_with_reciprocal_tariffs_%28cropped%29.jpg)

## Pendahuluan

Pada awal tahun 2025, kebijakan tarif impor baru yang dikenal sebagai **"Trump Tariffs"** diumumkan, menaikkan tarif impor barang ke Amerika Serikat secara signifikan dari sekitar 2,5% menjadi 27% [\[1\]](https://en.wikipedia.org/wiki/Tariffs_in_the_second_Trump_administration). Kebijakan ini berdampak pada dinamika ekonomi global, termasuk pasar saham di negara berkembang seperti Indonesia.

Indeks **LQ45** di Bursa Efek Indonesia mencerminkan performa 45 saham paling likuid dan menjadi indikator utama respon pasar domestik terhadap perubahan ekonomi global dan indikator makroekonomi seperti suku bunga acuan Bank Indonesia (BI Rate), inflasi, dan nilai tukar Rupiah terhadap Dolar AS [\[2\]](https://id.wikipedia.org/wiki/Indeks_LQ45).

Penelitian ini bertujuan mengukur pengaruh kebijakan **Trump Tariffs** terhadap pergerakan harga saham harian perusahaan dalam indeks LQ45. Variabel dependen berupa harga saham (Close price) harian, sedangkan variabel independen meliputi BI Rate, tingkat inflasi, nilai tukar USD/IDR, serta dummy penerapan tarif tersebut.

Model yang digunakan adalah **Generalized Linear Model (GLM)** dengan pendekatan panel data untuk menangani data longitudinal harga saham dari berbagai perusahaan. Pendekatan ini memungkinkan estimasi pengaruh variabel makro dan kebijakan eksternal terhadap harga saham dengan mempertimbangkan heterogenitas antar perusahaan dan hubungan data dalam waktu.

Dengan model ini, diharapkan diperoleh pemahaman yang lebih baik mengenai dampak kebijakan tarif impor AS terhadap pasar saham Indonesia secara empiris dan kuantitatif.

## Metodologi

### Sumber Data  
Penelitian ini menggunakan data sekunder yang diperoleh dari dua sumber utama: [Yahoo Finance](https://finance.yahoo.com) untuk data harga penutupan saham harian dan nilai tukar Rupiah terhadap Dolar AS, serta [situs resmi Bank Indonesia](https://www.bi.go.id) untuk data suku bunga acuan (BI Rate) dan tingkat inflasi. Sampel penelitian terdiri dari 45 perusahaan yang tergabung dalam Indeks LQ45 di Bursa Efek Indonesia dengan total observasi sebanyak 4050 data.

### Spesifikasi Model  
Analisis menggunakan **Generalized Linear Model (GLM)** dengan family **Gaussian** dan link function **identity**, yang secara ekuivalen merupakan regresi linier biasa. Model ini memodelkan variabel dependen berupa harga penutupan saham harian (`Close`) secara kontinu sebagai fungsi dari variabel-variabel makroekonomi dan dummy kebijakan tarif impor Amerika Serikat 2025 (**Trump Tariffs**), serta efek tetap untuk tiap perusahaan (`Ticker`) sebagai variabel kategori.

Model dapat dituliskan sebagai berikut:

$$
\text{Close}_{it} = \alpha + \sum_{j=1}^{44} \gamma_j \, \text{Ticker}_{ij} + \beta_1 \, \text{BI-Rate}_{t} + \beta_2 \, \text{Rate\ Inflasi}_{t} + \beta_3 \, \text{USD/IDR}_{t} + \beta_4 \, \text{Trump\ Tariffs}_{t} + \epsilon_{it}
$$

Keterangan:  
- $\text{Close}_{it}$: Harga penutupan saham perusahaan $i$ pada waktu $t$  
- $\text{Ticker}_{ij}$: Dummy variabel untuk perusahaan ke-$j$ (efek tetap perusahaan)  
- $\alpha$: Intersep model  
- $\beta_1, ..., \beta_4$: Koefisien regresi untuk variabel bebas makroekonomi dan kebijakan  
- $\epsilon_{it}$: Error term, diasumsikan berdistribusi normal dengan mean nol dan varians konstan

### Alasan Pemilihan Metode  
Metode GLM family Gaussian dengan link identity dipilih karena variabel target berupa harga saham harian bersifat kontinu dan diasumsikan memenuhi asumsi normalitas residual secara kasar. Model ini memungkinkan estimasi pengaruh variabel makroekonomi dan kebijakan tarif impor terhadap harga saham dengan memasukkan dummy variabel perusahaan untuk mengontrol perbedaan rata-rata harga saham antar perusahaan.

Meskipun data merupakan data panel dengan observasi berulang per perusahaan, pendekatan ini menggunakan efek tetap untuk tiap perusahaan tanpa pemodelan korelasi residual temporal secara eksplisit. Pendekatan ini cocok untuk analisis pengaruh rata-rata variabel independen terhadap harga saham harian secara keseluruhan.

Dengan pendekatan ini, penelitian dapat mengestimasi dampak kebijakan tarif impor Amerika Serikat 2025 dan variabel makroekonomi terhadap harga saham harian di Bursa Efek Indonesia pada perusahaan-perusahaan yang masuk Indeks LQ45.


# Hasil dan Pembahasan

## Import Library

In [2]:
pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.1 MB/s eta 0:00:00


In [21]:
# Library umum
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Pengambilan data finansial
import yfinance as yf

# Statistik dan ekonometrika
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

# Statistik tambahan
from scipy.stats import shapiro

## Data Loading

Mengambil data saham LQ45 dari laman Yahoo Finance

In [4]:
tickers = [
    "ACES.JK", "ADMR.JK", "ADRO.JK", "AKRA.JK", "AMMN.JK", "AMRT.JK", "ANTM.JK", "ARTO.JK",
    "ASII.JK", "BBCA.JK", "BBNI.JK", "BBRI.JK", "BBTN.JK", "BMRI.JK", "BRIS.JK", "BRPT.JK",
    "CPIN.JK", "CTRA.JK", "ESSA.JK", "EXCL.JK", "GOTO.JK", "ICBP.JK", "INCO.JK", "INDF.JK",
    "INKP.JK", "ISAT.JK", "ITMG.JK", "JPFA.JK", "JSMR.JK", "KLBF.JK", "MAPA.JK", "MAPI.JK",
    "MBMA.JK", "MDKA.JK", "MEDC.JK", "PGAS.JK", "PGEO.JK", "PTBA.JK", "SIDO.JK", "SMGR.JK",
    "SMRA.JK", "TLKM.JK", "TOWR.JK", "UNTR.JK", "UNVR.JK"
]

data = yf.download(tickers, start="2025-01-01", end="2025-05-31")['Close']

data_panel = data.stack().reset_index()
data_panel.columns = ['Date', 'Ticker', 'Close']

df_saham = data_panel.copy()
df_saham['Date'] = pd.to_datetime(df_saham['Date'])

df_saham

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  45 of 45 completed


,Date,Ticker,Close
0,2025-01-02,ACES.JK,755.000000
1,2025-01-02,ADMR.JK,1165.000000
2,2025-01-02,ADRO.JK,2530.000000
3,2025-01-02,AKRA.JK,1110.232544
4,2025-01-02,AMMN.JK,8475.000000
...,...,...,...
4090,2025-05-28,SMRA.JK,404.000000
4091,2025-05-28,TLKM.JK,2820.000000
4092,2025-05-28,TOWR.JK,585.000000
4093,2025-05-28,UNTR.JK,22450.000000


Mengambil data BI-rate dari laman resmi BI

In [5]:
df_birate = {
    'Date' : ['2024-12-18', '2025-01-15', '2025-02-19', '2025-03-19', '2025-04-23', '2025-05-21'],
    'BI-Rate' : [0.0600, 0.0575, 0.0575, 0.0575, 0.0575, 0.0550]
}

df_birate = pd.DataFrame(df_birate)
df_birate['Date'] = pd.to_datetime(df_birate['Date'])
df_birate['Start Date'] = df_birate['Date']
df_birate['End Date'] = df_birate['Date'].shift(-1)

last_end_date = df_saham['Date'].max()
df_birate.loc[df_birate.index[-1], 'End Date'] = last_end_date

df_birate

,Date,BI-Rate,Start Date,End Date
0,2024-12-18,0.0600,2024-12-18,2025-01-15
1,2025-01-15,0.0575,2025-01-15,2025-02-19
2,2025-02-19,0.0575,2025-02-19,2025-03-19
3,2025-03-19,0.0575,2025-03-19,2025-04-23
4,2025-04-23,0.0575,2025-04-23,2025-05-21
5,2025-05-21,0.0550,2025-05-21,2025-05-28


In [6]:
wide_df = df_saham.reset_index().pivot(index='Date', columns='Ticker', values='Close').reset_index()

def assign_value(date):
    for i in range(len(df_birate)):
        if df_birate.loc[i, 'Start Date'] <= date <= df_birate.loc[i, 'End Date']:
            return df_birate.loc[i, 'BI-Rate']
    return None

wide_df['BI-Rate'] = wide_df['Date'].apply(assign_value)
df_saham = wide_df.melt(id_vars=['Date', 'BI-Rate'], var_name='Ticker', value_name='Close')

df_saham

,Date,BI-Rate,Ticker,Close
0,2025-01-02,0.060,ACES.JK,755.0
1,2025-01-03,0.060,ACES.JK,755.0
2,2025-01-06,0.060,ACES.JK,755.0
3,2025-01-07,0.060,ACES.JK,735.0
4,2025-01-08,0.060,ACES.JK,705.0
...,...,...,...,...
4090,2025-05-22,0.055,UNVR.JK,1815.0
4091,2025-05-23,0.055,UNVR.JK,1785.0
4092,2025-05-26,0.055,UNVR.JK,1765.0
4093,2025-05-27,0.055,UNVR.JK,1740.0


Mengambil data tingkat inflasi dari laman resmi BI

In [7]:
df_inflasi = {
    'Date' : ['2025-01-01', '2025-02-01', '2025-03-01', '2025-04-01', '2025-05-01'],
    'Rate Inflasi' : [0.0160, 0.0195, 0.0103, -0.0009, 0.0076]
}

df_inflasi = pd.DataFrame(df_inflasi)
df_inflasi['Date'] = pd.to_datetime(df_inflasi['Date'])

df_inflasi

,Date,Rate Inflasi
0,2025-01-01,0.0160
1,2025-02-01,0.0195
2,2025-03-01,0.0103
3,2025-04-01,-0.0009
4,2025-05-01,0.0076


In [8]:
df_saham['Month'] = df_saham['Date'].dt.month
df_inflasi['Month'] = df_inflasi['Date'].dt.month
df_saham = df_saham.merge(df_inflasi[['Month', 'Rate Inflasi']], on='Month', how='left')
df_saham.drop(columns=['Month'], inplace=True)

Mengambil data rate nilai tukar dollar dari laman Yahoo Finance

In [9]:
kurs = yf.download(
    "USDIDR=X",
    start=wide_df['Date'].min().strftime('%Y-%m-%d'),
    end=(wide_df['Date'].max() + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
)

df_kurs = kurs[['Close']].reset_index()
df_kurs.columns = ['Date', 'USD_IDR']

df_kurs

[*********************100%***********************]  1 of 1 completed


,Date,USD_IDR
0,2025-01-02,16230.500000
1,2025-01-03,16201.099609
2,2025-01-06,16178.799805
3,2025-01-07,16179.099609
4,2025-01-08,16157.700195
...,...,...
98,2025-05-22,16347.599609
99,2025-05-23,16356.900391
100,2025-05-26,16211.000000
101,2025-05-27,16243.500000


In [10]:
df_saham = df_saham.merge(df_kurs, left_on='Date', right_on='Date', how='left')
df_saham

,Date,BI-Rate,Ticker,Close,Rate Inflasi,USD_IDR
0,2025-01-02,0.060,ACES.JK,755.0,0.0160,16230.500000
1,2025-01-03,0.060,ACES.JK,755.0,0.0160,16201.099609
2,2025-01-06,0.060,ACES.JK,755.0,0.0160,16178.799805
3,2025-01-07,0.060,ACES.JK,735.0,0.0160,16179.099609
4,2025-01-08,0.060,ACES.JK,705.0,0.0160,16157.700195
...,...,...,...,...,...,...
4090,2025-05-22,0.055,UNVR.JK,1815.0,0.0076,16347.599609
4091,2025-05-23,0.055,UNVR.JK,1785.0,0.0076,16356.900391
4092,2025-05-26,0.055,UNVR.JK,1765.0,0.0076,16211.000000
4093,2025-05-27,0.055,UNVR.JK,1740.0,0.0076,16243.500000


In [11]:
df_saham['Trump Tariffs'] = (df_saham.Date >= pd.to_datetime('2025-04-05')).astype(int)
df_saham

,Date,BI-Rate,Ticker,Close,Rate Inflasi,USD_IDR,Trump Tariffs
0,2025-01-02,0.060,ACES.JK,755.0,0.0160,16230.500000,0
1,2025-01-03,0.060,ACES.JK,755.0,0.0160,16201.099609,0
2,2025-01-06,0.060,ACES.JK,755.0,0.0160,16178.799805,0
3,2025-01-07,0.060,ACES.JK,735.0,0.0160,16179.099609,0
4,2025-01-08,0.060,ACES.JK,705.0,0.0160,16157.700195,0
...,...,...,...,...,...,...,...
4090,2025-05-22,0.055,UNVR.JK,1815.0,0.0076,16347.599609,1
4091,2025-05-23,0.055,UNVR.JK,1785.0,0.0076,16356.900391,1
4092,2025-05-26,0.055,UNVR.JK,1765.0,0.0076,16211.000000,1
4093,2025-05-27,0.055,UNVR.JK,1740.0,0.0076,16243.500000,1


In [12]:
df_saham.sort_values(by=['Ticker', 'Date'], inplace=True)
df_saham.set_index(['Ticker', 'Date'], inplace=True)
df_saham

BI-Rate   Close  Rate Inflasi       USD_IDR  Trump Tariffs
Ticker  Date                                                                  
ACES.JK 2025-01-02    0.060   755.0        0.0160  16230.500000              0
        2025-01-03    0.060   755.0        0.0160  16201.099609              0
        2025-01-06    0.060   755.0        0.0160  16178.799805              0
        2025-01-07    0.060   735.0        0.0160  16179.099609              0
        2025-01-08    0.060   705.0        0.0160  16157.700195              0
...                     ...     ...           ...           ...            ...
UNVR.JK 2025-05-22    0.055  1815.0        0.0076  16347.599609              1
        2025-05-23    0.055  1785.0        0.0076  16356.900391              1
        2025-05-26    0.055  1765.0        0.0076  16211.000000              1
        2025-05-27    0.055  1740.0        0.0076  16243.500000              1
        2025-05-28    0.055  1730.0        0.0076  16279.000000              1

[4095 rows x 5 columns]

## Model Development

Data Lengkap setelah disatukan

In [13]:
df_saham.dropna(inplace=True)
df_saham

BI-Rate   Close  Rate Inflasi       USD_IDR  Trump Tariffs
Ticker  Date                                                                  
ACES.JK 2025-01-02    0.060   755.0        0.0160  16230.500000              0
        2025-01-03    0.060   755.0        0.0160  16201.099609              0
        2025-01-06    0.060   755.0        0.0160  16178.799805              0
        2025-01-07    0.060   735.0        0.0160  16179.099609              0
        2025-01-08    0.060   705.0        0.0160  16157.700195              0
...                     ...     ...           ...           ...            ...
UNVR.JK 2025-05-22    0.055  1815.0        0.0076  16347.599609              1
        2025-05-23    0.055  1785.0        0.0076  16356.900391              1
        2025-05-26    0.055  1765.0        0.0076  16211.000000              1
        2025-05-27    0.055  1740.0        0.0076  16243.500000              1
        2025-05-28    0.055  1730.0        0.0076  16279.000000              1

[4050 rows x 5 columns]

Ringakasan dari data yang digunakan

In [14]:
df_saham.describe()

,BI-Rate,Close,Rate Inflasi,USD_IDR,Trump Tariffs
count,4050.000000,4050.000000,4050.000000,4050.000000,4050.000000
mean,0.057639,3495.027656,0.011171,16430.656630,0.355556
std,0.001011,4818.817578,0.006871,208.341001,0.478740
min,0.055000,64.000000,-0.000900,16157.700195,0.000000
25%,0.057500,945.209259,0.007600,16279.000000,0.000000
50%,0.057500,1895.000000,0.010300,16352.250000,0.000000
75%,0.057500,4000.000000,0.016000,16538.300781,1.000000
max,0.060000,26423.287109,0.019500,17051.900391,1.000000


Cek multikolinearitas dari variabel


In [15]:
X = df_saham.drop(columns=['Close',])
X = add_constant(X)

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_data.drop(index=0)

,Variable,VIF
1,BI-Rate,1.196004
2,Rate Inflasi,5.335810
3,USD_IDR,3.175236
4,Trump Tariffs,3.328826


Untuk memastikan tidak terjadi multikolinearitas antar variabel independen, dilakukan perhitungan Variance Inflation Factor (VIF). Hasil menunjukkan bahwa seluruh variabel memiliki nilai VIF di bawah ambang batas umum 10, yang menandakan tidak adanya masalah multikolinearitas serius. Nilai VIF tertinggi terdapat pada variabel `Rate Inflasi` ($5,33$), yang mengindikasikan adanya korelasi sedang dengan variabel lain, namun masih dalam batas toleransi. Sementara itu, variabel lainnya seperti `BI-Rate` ($1,19$), `USD_IDR` ($3,17$), `Trump Tariffs` ($3,32$), dan `Close_lag` ($1,00$) menunjukkan tidak adanya indikasi multikolinearitas. Dengan demikian, seluruh variabel dapat digunakan dalam pemodelan regresi panel dinamis tanpa perlu dilakukan transformasi atau penghapusan.

In [20]:
df_saham  = df_saham.reset_index()
model = smf.glm(formula='Close ~ Q("BI-Rate") + Q("Rate Inflasi") + USD_IDR + Q("Trump Tariffs") + C(Ticker)',
                data=df_saham,
                family=sm.families.Gaussian()).fit()

print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Close   No. Observations:                 4050
Model:                            GLM   Df Residuals:                     4001
Model Family:                Gaussian   Df Model:                           48
Link Function:               Identity   Scale:                      1.2898e+05
Method:                          IRLS   Log-Likelihood:                -29551.
Date:                Fri, 06 Jun 2025   Deviance:                   5.1603e+08
Time:                        14:06:19   Pearson chi2:                 5.16e+08
No. Iterations:                     3   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             7076.3446 

Model regresi menunjukkan bahwa variabel makroekonomi seperti `BI-Rate`, `inflasi`, dan `Trump Tariffs` memiliki dampak positif dan signifikan terhadap harga saham, sedangkan `nilai tukar USD/IDR` berdampak negatif secara signifikan. Misalnya, kenaikan 1 poin pada BI-Rate dihubungkan dengan peningkatan harga saham rata-rata sebesar $±54,860$ poin, sementara pelemahan rupiah sebesar 1 poin menyebabkan penurunan harga saham sebesar $±0,61$ poin. Selain itu, perbedaan antar saham tercermin dalam variabel dummy `C(Ticker)`, yang menunjukkan bahwa setiap saham memiliki baseline harga yang berbeda secara signifikan dibanding saham referensi. Nilai pseudo R-squared yang sangat tinggi (≈1) mengindikasikan bahwa model menjelaskan hampir seluruh variasi dalam harga saham, meskipun hal ini perlu dievaluasi lebih lanjut karena kemungkinan adanya overfitting akibat banyaknya parameter dalam model.


In [23]:
residuals = model.resid_response

In [26]:
stat, p = shapiro(residuals)
print(f"Shapiro-Wilk Test: stat={stat:.4f}, p-value={p:.4f}")
if p > 0.05:
    print("Residuals kemungkinan berdistribusi normal.")
else:
    print("Residuals kemungkinan tidak berdistribusi normal.")

Shapiro-Wilk Test: stat=0.8122, p-value=0.0000
Residuals kemungkinan tidak berdistribusi normal.


Hasil uji Shapiro-Wilk menunjukkan statistik sebesar $0,8122$ dengan p-value $0,0000$. Karena $p-value < 0,05$, maka dapat disimpulkan bahwa residual model **tidak berdistribusi normal** secara statistik. Hal ini menunjukkan adanya pelanggaran asumsi normalitas residual pada model GLM yang digunakan.

In [27]:
X = model.model.exog
bp_test = het_breuschpagan(residuals, X)
labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print(dict(zip(labels, bp_test)))

{'Lagrange multiplier statistic': np.float64(1709.6894514514097), 'p-value': np.float64(0.0), 'f-value': np.float64(60.89351670567999), 'f p-value': np.float64(0.0)}


- Statistik LM sebesar **1709.69** dengan p-value **0.0** (p < 0.05) menunjukkan adanya **autokorelasi yang signifikan** pada residual model.
- F-statistik sebesar **60.89** dengan p-value **0.0** juga menguatkan temuan ini.

Artinya, residual residual model tidak independen dari waktu (ada korelasi serial), sehingga asumsi independensi residual dilanggar. Pelanggaran ini bisa menyebabkan estimasi koefisien dan standar error menjadi tidak efisien dan inferensi menjadi kurang valid.

In [28]:
dw = durbin_watson(residuals)
print(f"Durbin-Watson stat: {dw:.3f}")

Durbin-Watson stat: 0.204


Nilai Durbin-Watson sebesar **0.204** menunjukkan adanya **autokorelasi positif yang sangat kuat** pada residual model. Karena nilai 0.204 sangat jauh dari 2, ini mengindikasikan residual residual saling berkorelasi positif secara signifikan.

# Kesimpulan

Model GLM yang dibangun menunjukkan hubungan signifikan antara variabel independen seperti BI-Rate, Rate Inflasi, USD_IDR, dan Trump Tariffs terhadap harga Close saham berbagai ticker. Namun, hasil uji asumsi menunjukkan adanya pelanggaran normalitas residual serta autokorelasi positif yang kuat pada residual model.

Pelanggaran asumsi ini berpotensi mempengaruhi validitas inferensi dan efisiensi estimasi koefisien. Oleh karena itu, disarankan untuk melakukan perbaikan model dengan mempertimbangkan transformasi data, penggunaan robust standard errors, atau model regresi yang dapat mengakomodasi autokorelasi residual agar hasil analisis menjadi lebih andal.

# Sumber

[[1](https://en.wikipedia.org/wiki/Tariffs_in_the_second_Trump_administration)] "Tariffs in the second Trump administration", Wikipedia, [Online]. Tersedia: https://en.wikipedia.org/wiki/Tariffs_in_the_second_Trump_administration. [Diakses: Jun. 5, 2025]. \
[[2](https://id.wikipedia.org/wiki/Indeks_LQ45)] "Indeks LQ45", Wikipedia, [Online]. Tersedia: https://id.wikipedia.org/wiki/Indeks_LQ45. [Diakses: Jun. 5, 2025]. \
[3] E. W. Frees, Longitudinal and Panel Data: Analysis and Applications in the Social Sciences. Cambridge: Cambridge University Press, 2004.